In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('test.csv')
data.tail(2)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal
1458,2919,60,RL,74.0,9627,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,11,2006,WD,Normal


In [3]:
# 변수 1~16
def fe1(df):
    temp = df.copy()
    
    fre_D = temp['LotFrontage'].value_counts().idxmax()  # 최빈값
    temp['LotFrontage'].fillna(fre_D, inplace=True)
    
    temp = temp.drop('Alley', axis=1)
    temp = temp.drop('Utilities', axis=1)
    
    temp['MSSubClass'].loc[temp['MSSubClass'].isin([20,60,120])] = 'new'
    temp['MSSubClass'].loc[temp['MSSubClass'].isin([30,70,160])] = 'older'
    temp['MSSubClass'].loc[temp['MSSubClass'].isin([40,45])] = 'unf'
    temp['MSSubClass'].loc[temp['MSSubClass'].isin([80,85,180])] = 'split'
    temp['MSSubClass'].loc[temp['MSSubClass'].isin([50,75,90,150,190])] = 'other'
    
    temp['LotShape'].loc[temp.LotShape == 'Reg'] = 'reg'
    temp['LotShape'].loc[temp['LotShape'].isin(['IR1','IR2','IR3'])] = 'ir'
    
    temp['LandSlope'].loc[temp.LotShape == 'Gtl'] = 'Gtl'
    temp['LandSlope'].loc[temp['LandSlope'].isin(['Sev','Mod'])] = 'ste'
    
    temp['Condition1'].loc[temp.LotShape == 'Norm'] = 'Norm'
    temp['Condition1'].loc[temp['Condition1'].isin(['PosN','PosA'])] = 'pos'
    temp['Condition1'].loc[temp['Condition1']\
                        .isin(['Feedr','Artery','RRAe','RRNn','RRAn','RRNe'])] = 'nag'

    temp['HouseStyle'].loc[temp['HouseStyle'].isin(['1Story','1.5Fin'])] = 'sto1'
    temp['HouseStyle'].loc[temp['HouseStyle'].isin(['2Story','SFoyer','SLvl','2.5Fin'])] = 'sto2'
    temp['HouseStyle'].loc[temp['HouseStyle'].isin(['2.5Unf','1.5Unf'])] = 'unf'
    
    return temp

# 변수 17~32
def fe2(df):
    temp = df.copy()
    
    temp['MasVnrArea'].fillna(0, inplace=True)
    temp['MasVnrType'].fillna('None', inplace=True)
    temp['BsmtQual'].fillna('Na', inplace=True)
    temp['BsmtCond'].fillna('Na', inplace=True)
    temp['BsmtExposure'].fillna('Na', inplace=True)
    
    temp['RoofStyle'] = temp['RoofStyle']\
        .map({'Gable':'Gable', 'Hip':'Hip', 'Flat':'Others',
          'Gambrel':'Others', 'Mansard':'Others','Shed':'Others'})
    
    temp['RoofMatl'] = temp['RoofMatl']\
        .map({'CompShg':'CompShg', 'WdShngl':'Wood','WdShake':'Wood', 'Metal':'Others',
          'Membran':'Others','Tar&Grv':'Others', 'Roll':'Others','ClyTile':'Others'})
    
    temp['Exterior1st'] = temp['Exterior1st']\
        .map({'VinylSd':'VinylSd', 'HdBoard':'HdBoard','MetalSd':'MetalSd','Wd Sdng':'Wood',
              'WdShing':'Wood','Plywood':'Panull', 'CemntBd':'Panull','BrkComm':'Brick',
              'BrkFace':'Brick','CBlock':'Brick','Stone':'Brick', 'PreCast': 'Brick',
              'AsphShn':'Shn', 'AsbShng':'Shn','ImStucc':'Other','Stucco':'Other','Other':'Other'})
    
    temp['Exterior2nd'] = temp['Exterior2nd']\
        .map({'VinylSd':'VinylSd', 'HdBoard':'HdBoard','MetalSd':'MetalSd',
          'Wd Sdng':'Wood', 'Wd Shng':'Wood','Plywood':'Panull',
          'CmentBd':'Panull','Brk Cmn':'Brick','BrkFace':'Brick',
          'CBlock':'Brick','Stone':'Brick', 'PreCast': 'Brick', 'AsphShn':'Shn','AsbShng':'Shn',
          'ImStucc':'Other','Stucco':'Other','Other':'Other'})
    
    # 지어진 년도: 1800 후반, ~1950, ~2000, 2000~
    bin = [-np.inf, 1900, 1951, 2001, np.inf]
    label = ['1800 후반', '~1950', '~2000', '2000~']
    temp['YearBuilt_grp'] = pd.cut(temp['YearBuilt'], bins=bin, right=False, labels=label)
    # 기존 변수 제거
    temp.drop('YearBuilt', axis=1, inplace=True)
    
    # 리모델링 년도: ~1950, ~2000, 2000~
    bin = [-np.inf, 1951, 2001, np.inf]
    label = ['~1950', '~2000', '2000~']
    temp['YearRemodAdd_grp'] = pd.cut(temp['YearRemodAdd'], bins=bin, right=False, labels=label)
    # 기존 변수 제거
    temp.drop('YearRemodAdd', axis=1, inplace=True)
    
    return temp

# 변수 33~48 전처리
def fe3(df):
    temp = df.copy()
    
    temp['BsmtFinType1'].fillna("NA", inplace = True)
    temp['BsmtFinType2'].fillna("NA", inplace = True)

    ## 최빈값으로 채움
    temp['Electrical'].fillna(temp['Electrical'].mode()[0], inplace = True)
    
    bins = [-np.inf, 1, 500, 1000, 1500, np.inf]
    labels = ['없음', '500미만', '1000미만', '1500미만', '1500이상']
    temp['BsmtFinSF1_grp'] = pd.cut(temp['BsmtFinSF1'], bins, labels=labels, right=False)
    
    bins = [-np.inf, 1, 500, 1000, 1500, 2000, np.inf]
    labels = ['없음', '500미만', '1000미만', '1500미만', '2000미만', '2000이상']
    temp['BsmtUnfSF_grp'] = pd.cut(temp['BsmtUnfSF'], bins, labels=labels, right=False)    
    
    return temp

# 변수 49~64 전처리
def fe4(df):
    temp = df.copy()
    
    temp['FireplaceQu'].fillna('No', inplace=True)
    temp['GarageType'].fillna('No', inplace=True)
    temp['GarageYrBlt'].fillna(0, inplace=True)
    temp['GarageFinish'].fillna('No', inplace=True)
    temp['GarageQual'].fillna('No', inplace=True)
    temp['GarageCond'].fillna('No', inplace=True)
    
    temp['KitchenAbvGr'] = temp['KitchenAbvGr'].map({ 1:'1개', 2:'2개 이상', 3:'2개 이상'})
    
    bin = [1, 5, 9, np.inf]
    label = ['~4', '5~8', '9~']
    temp['TotRmsAbvGrd'] = pd.cut(temp['TotRmsAbvGrd'], bins=bin, labels=label, right=False)
    
    temp['Functional'] = temp['Functional']\
        .map({'Min1':'Min', 'Min2':'Min', 'Maj1':'Maj', 'Maj2':'Maj', 'Mod':'Typ','Typ':'Typ',
             'Sev': 'Sev', 'Sal': 'Sal'})
    
    temp['GarageType'] = temp['GarageType']\
        .map({'Attchd':'At', 'Detchd':'Dt', 'BuiltIn':'Others', 'CarPort':'Others',
              'Basment':'Others', '2Types':'Others', 'No':'No'})
    
    bin = [-np.inf, 1, 1910, 1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000, np.inf]
    label = ['No', '1900년대', '1910년대', '1920년대', '1930년대',
             '1940년대', '1950년대', '1960년대', '1970년대', '1980년대', '1990년대', '2000년대']
    temp['GarageYrBlt'] = pd.cut(temp['GarageYrBlt'], bins=bin, labels=label, right=False)
    
    temp['GarageCond'] = temp['GarageCond']\
        .map({'TA':'TA', 'No':'No', 'Ex':'Gd','Gd':'Gd','Fa':'Po','Po':'Po'})
    
    return temp

# 변수 65~80 전처리
def fe5(df):
    temp = df.copy()
        
    # 변수 65~80 전처리
    temp['PoolQC'].fillna('NO', inplace=True)
    temp['Fence'].fillna('NO', inplace=True)
    temp['MiscFeature'].fillna('NO', inplace=True)
    
    bin = [-np.inf, 1, 200, np.inf]
    label = ['No', '200미만', '200이상']
    temp['WoodDeckSF_cut'] = pd.cut(temp['WoodDeckSF'], bins=bin, labels=label, right=False)
    
    bin = [-np.inf, 1, 100, np.inf]
    label = ['No', '100미만', '100이상']
    temp['OpenPorchSF_cut'] = pd.cut(temp['OpenPorchSF'], bins=bin, labels=label, right=False)
    
    bin = [-np.inf, 1, 100, 200, 300, np.inf]
    label = ['No', '100미만', '200미만', '300미만', '300이상']
    temp['EnclosedPorch_cut'] = pd.cut(temp['EnclosedPorch'], bins=bin, labels=label, right=False)
    
    return temp

def del_fe(df):
    temp = df.copy()
    
    # 이변량 분석을 통한 변수 제거
    cols = ['Street', 'LandSlope', 'Condition2', 'OverallCond', 'BsmtFinSF2', 'LowQualFinSF',
            '3SsnPorch', 'PoolArea', 'MiscFeature', 'MiscVal', 'MoSold', 'YrSold']
    temp.drop(cols, axis=1, inplace=True)
    
    # 다중공선성을 위한 변수명 변경(밑에서 제거)
    temp.rename(columns={'1stFlrSF':'firstFlrSF', '2ndFlrSF':'secondFlrSF'}, inplace=True)
    
    # 다중공선성 변수 제거
    cols = ['firstFlrSF', 'secondFlrSF']
    temp.drop(cols, axis=1, inplace=True)
    
    # 고유값 변수 제거
    temp.drop('Id', axis=1, inplace=True)
    
    return temp

def dumm_fe(df, cat):
    temp = df.copy()
    
    # category로 타입변경
    for k, v in cat.items():
        temp[k] = pd.Categorical(temp[k], categories=v, ordered=False)
    
    # 가변수화
    cat_cols = [col for col in temp.columns if temp[col].dtype != 'float64' if temp[col].dtype != 'int64']
    temp = pd.get_dummies(temp, columns=cat_cols, drop_first=True)
    
    return temp

def df(df):
    temp = df.copy()

    x_cols = list(temp)

    return pd.DataFrame(temp, columns = x_cols)

## 오브젝트 불러오기

In [4]:
import joblib

In [5]:
cat = joblib.load('cat.pkl')

lr = joblib.load('model_lr.pkl')
xgbr = joblib.load('model_xgbr.pkl')

## 전처리 함수 적용

In [6]:
data = fe1(data)
data = fe2(data)
data = fe3(data)
data = fe4(data)
data = fe5(data)
data = del_fe(data)

C:\Users\User\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


## 결측치 이슈

In [7]:
tmp = data.isna().sum()
print(tmp[tmp>0])

MSZoning          4
Exterior1st       1
Exterior2nd       1
BsmtFinSF1        1
BsmtUnfSF         1
TotalBsmtSF       1
BsmtFullBath      2
BsmtHalfBath      2
KitchenAbvGr      2
KitchenQual       1
Functional        2
GarageCars        1
GarageArea        1
SaleType          1
BsmtFinSF1_grp    1
BsmtUnfSF_grp     1
dtype: int64


## 결측치 채우기

In [8]:
from sklearn.impute import SimpleImputer

In [9]:
imputer_list = ['MSZoning', 'Exterior1st', 'Exterior2nd', 'BsmtFullBath', 'BsmtHalfBath', 'KitchenAbvGr',
                'KitchenQual', 'Functional', 'GarageCars', 'SaleType', 'BsmtFinSF1_grp', 'BsmtUnfSF_grp']
imputer = SimpleImputer(strategy = 'most_frequent')
data[imputer_list] = imputer.fit_transform(data[imputer_list])

In [10]:
imputer_list2 = ['BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF', 'GarageArea']
imputer2 = SimpleImputer(strategy = 'mean')
data[imputer_list2] = imputer2.fit_transform(data[imputer_list2])

## 가변수화 및 df화

In [11]:
data = dumm_fe(data, cat)
data = df(data)
data.tail(2)

,LotFrontage,LotArea,OverallQual,MasVnrArea,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,GrLivArea,BsmtFullBath,BsmtHalfBath,...,BsmtUnfSF_grp_500미만,BsmtUnfSF_grp_없음,WoodDeckSF_cut_200미만,WoodDeckSF_cut_200이상,OpenPorchSF_cut_100미만,OpenPorchSF_cut_100이상,EnclosedPorch_cut_100미만,EnclosedPorch_cut_200미만,EnclosedPorch_cut_300미만,EnclosedPorch_cut_300이상
1457,62.0,10441,5,0.0,337.0,575.0,912.0,970,0.0,1.0,...,0,0,1,0,1,0,0,0,0,0
1458,74.0,9627,7,94.0,758.0,238.0,996.0,2000,0.0,0.0,...,1,0,1,0,1,0,0,0,0,0


In [12]:
data.shape

(1459, 233)

In [13]:
lr_pred = lr.predict(data)

C:\Users\User\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- BsmtFinSF1_grp_없음
- BsmtUnfSF_grp_없음
Feature names seen at fit time, yet now missing:
- BsmtFinSF1_grp_1000미만
- BsmtUnfSF_grp_1000미만

  warnings.warn(message, FutureWarning)


In [18]:
len(lr_pred)

1459

In [20]:
lr_pred = pd.Series(lr_pred)
lr_pred

0       133789.843974
1       193189.717871
2       221744.260538
3       212491.203602
4       201555.349036
            ...      
1454     74218.189284
1455     85312.896626
1456    191584.511021
1457    102219.613021
1458    229266.503112
Length: 1459, dtype: float64

In [22]:
lr_pred.to_csv('predict_lr.csv')

In [15]:
xgbr_pred = xgbr.predict(data)

In [17]:
len(xgbr_pred)

1459

In [23]:
xgbr_pred = pd.Series(xgbr_pred)
xgbr_pred

0       143791.796875
1       149373.656250
2       173373.812500
3       188055.640625
4       212082.234375
            ...      
1454     77032.632812
1455     75705.156250
1456    123624.718750
1457    119919.789062
1458    215451.343750
Length: 1459, dtype: float32

In [24]:
xgbr_pred.to_csv('predict_xgbr.csv')